<a href="https://colab.research.google.com/github/Emanuel3queijos/AWS-University-ETL/blob/main/python_scripts/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicializar bibliotecas

In [ ]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt


--2024-11-10 17:43:33--  https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt
Resolving emanuel3queijos.github.io (emanuel3queijos.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to emanuel3queijos.github.io (emanuel3queijos.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     691  --.-KB/s    in 0s      

2024-11-10 17:43:33 (37.6 MB/s) - ‘requirements.txt’ saved [691/691]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.

# Scripts para adicionar data dentro do banco



In [ ]:
import psycopg2

# Função para criar a conexão
def create_connection():
    try:
        conn = psycopg2.connect(
            host="db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com",
            dbname="AWS-University-ETL-DB",
            user="postgres",
            password="Emanu0710**",
            port=5432
        )

        return conn
    except Exception as error:
        print(f"Erro ao conectar ao banco de dados: {error}")
        return None
create_connection()

<connection object at 0x7f1508abf380; dsn: 'user=postgres password=xxx dbname=AWS-University-ETL-DB host=db-manu-etluniversityproject-instance.c44vb5orqds7.us-east-1.rds.amazonaws.com port=5432', closed: 0>

In [ ]:
def inserir_departamentos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Inserir departamentos
            departamentos = [
                ("Escola de Tecnologia", "Departamento focado em áreas tecnológicas."),
                ("Escola de Engenharia", "Departamento voltado para as engenharias."),
                ("Escola de Saúde", "Departamento voltado para ciências da saúde."),
                ("Ciências Biológicas", "Departamento voltado para biologia e afins.")
            ]
            for nome, descricao in departamentos:
                cur.execute("""
                    INSERT INTO tb_departamento (nome_departamento, descricao)
                    VALUES (%s, %s)
                    RETURNING cod_departamento
                """, (nome, descricao))
                cod_departamento = cur.fetchone()[0]
                print(f"Departamento {nome} inserido com sucesso. Código: {cod_departamento}")

            conn.commit()
            print("Departamentos e inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_departamentos()


Departamento Escola de Tecnologia inserido com sucesso. Código: 10
Departamento Escola de Engenharia inserido com sucesso. Código: 11
Departamento Escola de Saúde inserido com sucesso. Código: 12
Departamento Ciências Biológicas inserido com sucesso. Código: 13
Departamentos e inseridos com sucesso.


In [ ]:
def inserir_cursos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Dicionário onde as chaves são os nomes dos departamentos
            cursos_por_departamento = {
                "Escola de Tecnologia": [
                    ("Engenharia de Software", "Curso voltado ao desenvolvimento de software.", 4, 1200.00),
                    ("Ciência da Computação", "Foco em computação e sistemas.", 4, 1150.00),
                    ("Inteligência Artificial", "Curso para aprendizado de IA.", 4, 1300.00),
                    ("Redes de Computadores", "Infraestrutura e comunicação de redes.", 3, 1100.00)
                ],
                "Escola de Engenharia": [
                    ("Engenharia Civil", "Foco em obras e infraestrutura.", 5, 1400.00),
                    ("Engenharia Elétrica", "Curso de sistemas elétricos.", 5, 1350.00),
                    ("Engenharia Mecânica", "Curso sobre mecânica aplicada.", 5, 1380.00),
                    ("Engenharia de Controle e Automação", "Automação industrial.", 5, 1450.00)
                ],
                "Escola de Saúde": [
                    ("Medicina", "Foco em prática médica.", 6, 2000.00),
                    ("Enfermagem", "Prática de enfermagem.", 4, 1100.00),
                    ("Farmácia", "Estudos sobre medicamentos.", 4, 1250.00),
                    ("Biomedicina", "Ciência biomédica.", 4, 1200.00)
                ],
                "Ciências Biológicas": [
                    ("Biologia", "Estudo da vida.", 4, 1100.00),
                    ("Ecologia", "Curso sobre ecossistemas.", 4, 1050.00),
                    ("Zoologia", "Estudo dos animais.", 4, 1100.00),
                    ("Botânica", "Estudo das plantas.", 4, 1000.00)
                ]
            }

            for nome_departamento, cursos in cursos_por_departamento.items():
                # Busca o código do departamento pelo nome
                cur.execute("""
                    SELECT cod_departamento FROM tb_departamento WHERE nome_departamento = %s
                """, (nome_departamento,))
                cod_departamento = cur.fetchone()

                if cod_departamento is None:
                    print(f"Departamento '{nome_departamento}' não encontrado.")
                    continue

                cod_departamento = cod_departamento[0]

                for nome_curso, descricao_curso, duracao, mensalidade in cursos:
                    cur.execute("""
                        INSERT INTO tb_curso (nome_curso, descricao, cod_departamento, duracao, valor_mensalidade)
                        VALUES (%s, %s, %s, %s, %s)
                        RETURNING cod_curso
                    """, (nome_curso, descricao_curso, cod_departamento, duracao, mensalidade))
                    cod_curso = cur.fetchone()[0]

                    print(f"Curso {nome_curso} inserido com sucesso. Código: {cod_curso}")

            conn.commit()
            print("Departamentos, cursos e disciplinas inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_cursos()


In [ ]:
from faker import Faker
import psycopg2

fake = Faker('pt_BR')

def gerar_disciplinas_por_semestre():
    disciplinas = {}
    for semestre in range(1, 9):  # 8 semestres
        disciplinas[semestre] = []
        for _ in range(6):  # 6 disciplinas por semestre
            nome = fake.sentence(nb_words=3)
            descricao = fake.text(max_nb_chars=50)
            carga_horaria = fake.random_int(min=60, max=80)
            disciplina = {
                "nome": nome,
                "descricao": descricao,
                "carga_horaria": carga_horaria
            }
            disciplinas[semestre].append(disciplina)
    return disciplinas

def inserir_disciplinas():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            # Disciplinas geradas aleatoriamente para cada semestre
            disciplinas_por_semestre = gerar_disciplinas_por_semestre()

            departamentos = {
                "Escola de Tecnologia": 10,
                "Escola de Engenharia": 11,
                "Escola de Saúde": 12,
                "Ciências Biológicas": 13
            }

            for nome_departamento, cod_departamento in departamentos.items():
                cur.execute("SELECT cod_curso FROM tb_curso WHERE cod_departamento = %s", (cod_departamento,))
                cursos = cur.fetchall()

                for curso in cursos:
                    cod_curso = curso[0]

                    for semestre, disciplinas in disciplinas_por_semestre.items():
                        for disciplina_data in disciplinas:
                            nome_disciplina = disciplina_data["nome"]
                            descricao_disciplina = disciplina_data["descricao"]
                            carga_horaria = disciplina_data["carga_horaria"]

                            cur.execute("""
                                INSERT INTO tb_disciplina (nome_disciplinas, descricao, carga_horaria, semestre_oferecido, cod_departamento)
                                VALUES (%s, %s, %s, %s, %s)
                                RETURNING cod_disciplina
                            """, (nome_disciplina, descricao_disciplina, carga_horaria, semestre, cod_departamento))
                            cod_disciplina = cur.fetchone()[0]

                            cur.execute("""
                                INSERT INTO tb_curso_disciplina (cod_departamento, cod_curso, cod_disciplina)
                                VALUES (%s, %s, %s)
                            """, (cod_departamento, cod_curso, cod_disciplina))

                            print(f"Disciplina '{nome_disciplina}' inserida com sucesso no curso com código '{cod_curso}' no departamento '{nome_departamento}'.")

            conn.commit()
            print("Disciplinas inseridas com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_disciplinas()


Disciplina 'Quo at magni.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Quis doloremque numquam.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Libero maiores.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Aliquid assumenda.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Expedita.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Numquam quam.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Earum facilis.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Ut aliquam.' inserida com sucesso no curso com código '3' no departamento 'Escola de Tecnologia'.
Disciplina 'Aperiam aliquam quod.' inserida com sucesso no curso com código '3' no departam

In [ ]:
from os import name
from faker import Faker
import random

fake = Faker()

def inserir_professores():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Consultar disciplinas e departamentos para alocar professores
            cur.execute("SELECT cod_disciplina, cod_departamento FROM tb_disciplina")
            disciplinas = cur.fetchall()

            for cod_disciplina, cod_departamento in disciplinas:
                # Definir aleatoriamente de 1 a 3 professores para a disciplina
                num_professores = random.randint(1, 2)

                for _ in range(num_professores):
                    # Gerar dados fictícios para o professor
                    lastname = fake.last_name()
                    nome = fake.name()+ " " + lastname
                    data_nascimento = fake.date_of_birth(minimum_age=30, maximum_age=60)
                    email = fake.email().split('@')[0] + f'.{lastname}@email.com'
                    telefone = fake.phone_number()[:15]
                    endereco = fake.address()
                    matricula = fake.unique.random_number(digits=20)
                    cpf = fake.unique.numerify('###########')  # Gera um CPF com 11 dígitos

                    cur.execute("""
                        INSERT INTO tb_professor (nome_professor, data_nascimento, email, cod_departamento, telefone, endereco, matricula, cpf)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                        RETURNING cod_professor
                    """, (nome, data_nascimento, email, cod_departamento, telefone, endereco, matricula, cpf))

                    cod_professor = cur.fetchone()[0]  # Recuperar o ID do professor inserido

                    # Inserir associação do professor com a disciplina
                    cur.execute("""
                        INSERT INTO prof_disciplina (cod_professor, cod_disciplina)
                        VALUES (%s, %s)
                    """, (cod_professor, cod_disciplina))

                    print(f"Professor '{nome}' alocado na disciplina de código '{cod_disciplina}', CPF '{cpf}', email: {email}.")

            conn.commit()
            print("Professores inseridos e alocados com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir professores: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

# Executar o script
inserir_professores()


Professor 'Hunter Williams Williams' alocado na disciplina de código '74', CPF '28049555785', email: torresdennis.Williams@email.com.
Professor 'Erin Melton Jones' alocado na disciplina de código '75', CPF '98434781142', email: rchoi.Jones@email.com.
Professor 'Jimmy Gates Gonzalez' alocado na disciplina de código '76', CPF '96681806342', email: dshannon.Gonzalez@email.com.
Professor 'Cody Richard Jones' alocado na disciplina de código '76', CPF '16834742536', email: bmckay.Jones@email.com.
Professor 'Jaime Williams Smith' alocado na disciplina de código '77', CPF '60730937678', email: amandawalter.Smith@email.com.
Professor 'Robert Farrell Blackburn' alocado na disciplina de código '77', CPF '11360565750', email: smithlaura.Blackburn@email.com.
Professor 'Jerry Gibbs Lam' alocado na disciplina de código '78', CPF '57810086911', email: reginaldwalker.Lam@email.com.
Professor 'Daniel Alvarado Potter' alocado na disciplina de código '78', CPF '69016535710', email: audreyrodriguez.Potter@

In [46]:
from faker import Faker
import random
import string

fake = Faker()

def gerar_sigla_turma():
    return ''.join(random.choices(string.ascii_uppercase, k=3))

def gerar_sala():
    return f"{random.choice(string.ascii_uppercase)}{random.randint(100, 399)}"

def inserir_turmas():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            # Consultar disciplinas com seus semestres e departamentos para alocar turmas
            cur.execute("SELECT cod_disciplina, cod_departamento, semestre_oferecido FROM tb_disciplina")
            disciplinas = cur.fetchall()

            for cod_disciplina, cod_departamento, semestre_oferecido in disciplinas:

                # Obter o código do semestre a partir de tb_semestre usando semestre_oferecido
                cur.execute("SELECT cod_semestre FROM tb_semestre WHERE numero = %s", (semestre_oferecido,))
                cod_semestre_result = cur.fetchone()

                if cod_semestre_result:
                    cod_semestre = cod_semestre_result[0]

                    # Listar as faixas de semestres: 1º e 2º, 3º e 4º, 5º e 6º, 7º e 8º
                    faixas_semestres = [(1, 2), (3, 4), (5, 6), (7, 8)]

                    for ano_turma in range(1990, 2000):  # Período de 10 anos
                        for semestre_par, semestre_impar in faixas_semestres:
                            for _ in range(3):  # 3 turmas por semestre em cada faixa

                                # Buscar professores do mesmo departamento
                                cur.execute("""
                                    SELECT cod_professor FROM tb_professor
                                    WHERE cod_departamento = %s
                                """, (cod_departamento,))
                                professores = cur.fetchall()

                                if professores:
                                    num_professores = random.randint(1, min(3, len(professores)))
                                    professores_escolhidos = random.sample(professores, num_professores)

                                    turno = random.choice(['Manhã', 'Tarde', 'Noite'])
                                    sala = gerar_sala()
                                    horario = fake.time(pattern="%H:%M")
                                    nome_turma = gerar_sigla_turma()

                                    # Inserir a turma com todos os atributos necessários
                                    cur.execute("""
                                        INSERT INTO tb_turma (cod_disciplina, cod_professor, nome_turma, ano, cod_semestre, turno, sala, horario)
                                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                                        RETURNING cod_turma
                                    """, (cod_disciplina, professores_escolhidos[0][0], nome_turma, ano_turma, cod_semestre, turno, sala, horario))

                                    cod_turma_result = cur.fetchone()
                                    if cod_turma_result:
                                        cod_turma = cod_turma_result[0]
                                    else:
                                        raise ValueError("Falha ao obter o código da turma inserida.")

                                    print(f"Turma '{nome_turma}' criada para disciplina {cod_disciplina}, ano {ano_turma}, faixa de semestre {semestre_par} e {semestre_impar}. Professores: {[p[0] for p in professores_escolhidos]}.")

            conn.commit()
            print("Turmas inseridas com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir turmas: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_turmas()


Streaming output truncated to the last 5000 lines.
Turma 'NTK' criada para disciplina 800, ano 1993, faixa de semestre 3 e 4. Professores: [5377, 5187].
Turma 'FKJ' criada para disciplina 800, ano 1993, faixa de semestre 5 e 6. Professores: [5373, 5140].
Turma 'LHT' criada para disciplina 800, ano 1993, faixa de semestre 5 e 6. Professores: [5269, 5306, 5322].
Turma 'EBV' criada para disciplina 800, ano 1993, faixa de semestre 5 e 6. Professores: [5297, 5319, 5161].
Turma 'UHT' criada para disciplina 800, ano 1993, faixa de semestre 7 e 8. Professores: [5146].
Turma 'LIQ' criada para disciplina 800, ano 1993, faixa de semestre 7 e 8. Professores: [5349].
Turma 'LMG' criada para disciplina 800, ano 1993, faixa de semestre 7 e 8. Professores: [5093].
Turma 'SAP' criada para disciplina 800, ano 1994, faixa de semestre 1 e 2. Professores: [5140].
Turma 'WLH' criada para disciplina 800, ano 1994, faixa de semestre 1 e 2. Professores: [5328].
Turma 'ZRE' criada para disciplina 800, ano 1994,

In [ ]:
from faker import Faker
import random

fake = Faker()

def inserir_alunos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
## preciso ir dormir

            conn.commit()
            print("Alunos inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir alunos: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_alunos()


Erro ao inserir alunos: integer division or modulo by zero


In [ ]:

def export_to_csv():
    conn = create_connection()
    try:
        cur = conn.cursor()

        tables = ["tb_aluno", "tb_curso", "tb_disciplina", "tb_turma", "tb_matricula", "tb_inscricao", "tb_turma_aluno"]
        for table in tables:
            with open(f'{table}.csv', 'w') as f:
                cur.copy_expert(f"COPY {table} TO STDOUT WITH CSV HEADER", f)

        cur.close()
        print("Dados exportados com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao exportar dados: {error}")
    finally:
        if conn is not None:
            conn.close()


#Export tables
export_to_csv()

In [ ]:
# Limpar as tabelas
def clean_tables(conn):
    conn = create_connection()
    try:
        cur = conn.cursor()
        truncate_sql = """TRUNCATE TABLE tb_turma_aluno, tb_inscricao, tb_matricula, tb_turma, tb_disciplina, tb_curso, tb_aluno RESTART IDENTITY CASCADE;"""
        cur.execute(truncate_sql)
        conn.commit()
        print("Tabelas limpas com sucesso")
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro ao truncar tabelas: {error}")

    finally:
        if conn is not None:
            conn.close()

clean_tables()

Tabelas limpas com sucesso
